In [1]:
# prompt: open google drive

from google.colab import drive
drive.mount('/content/drive')
import os


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
base_path="/content/drive/MyDrive/Machine\ Visual\ Perception/3D-GS/gaussian-splatting/colab"
%cd $base_path
# !cd ..
print(os.getcwd())
!git status

/content/drive/MyDrive/Machine Visual Perception/3D-GS/gaussian-splatting/colab
/content/drive/MyDrive/Machine Visual Perception/3D-GS/gaussian-splatting/colab
Refresh index: 100% (62/62), done.
On branch mario-dev
Your branch is ahead of 'origin/mario-dev' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   3d-gs.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	cloudflared-linux-amd64.deb
	../submodules/


It took 2.63 seconds to enumerate untracked files. 'status -uno'
may speed it up, but you have to be careful not to forget to add
new files yourself (see 'git help status').
no changes added to commit (use "git add" and/or "git commit -a")


In [3]:
!git config --global user.email "mariopariona117@gmail.com"
!git config --global user.name "Mario"

In [4]:
!mkdir -p ~/.ssh
!cp "/content/drive/My Drive/ssh_keys/id_ed25519" ~/.ssh/
!cp "/content/drive/My Drive/ssh_keys/id_ed25519.pub" ~/.ssh/
!chmod 600 ~/.ssh/id_ed25519
!eval "$(ssh-agent -s)" && ssh-add ~/.ssh/id_ed25519

Agent pid 3137
Identity added: /root/.ssh/id_ed25519 (mariopariona117@gmail.com)


In [ ]:
base_path="/content/drive/MyDrive/Machine\ Visual\ Perception/3D-GS/gaussian-splatting/colab"
%cd $base_path
# !git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q plyfile

# %cd $base_path/gaussian-splatting
# !pip install -q $base_path/gaussian-splatting/submodules/diff-gaussian-rasterization
# !pip install -q $base_path/gaussian-splatting/submodules/simple-knn
#
# !wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
# !unzip tandt_db.zip
#
# !python train.py -s tandt/train

# !wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/GaussianViewTest.zip
# !unzip GaussianViewTest.zip
# !python render.py -m /content/gaussian-splatting/GaussianViewTest/model
# !ffmpeg -framerate 3 -i /content/gaussian-splatting/GaussianViewTest/model/train/ours_30000/renders/%05d.png -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -r 3 -pix_fmt yuv420p /content/renders.mp4
# !ffmpeg -framerate 3 -i /content/gaussian-splatting/GaussianViewTest/model/train/ours_30000/gt/%05d.png -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -r 3 -pix_fmt yuv420p /content/gt.mp4 -y

/content/drive/MyDrive/Machine Visual Perception/3D-GS/gaussian-splatting/colab


In [ ]:
%cd $base_path
print(os.getcwd())
!git clone https://github.com/antimatter15/splat $base_path/splat
import os

!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -O $base_path/cloudflared-linux-amd64.deb
!dpkg -i $base_path/cloudflared-linux-amd64.deb

import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['cloudflared', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(7860, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)
%cd $base_path/splat
!python -m http.server 7860

/content/drive/MyDrive/Machine Visual Perception/3D-GS/gaussian-splatting/colab
/content/drive/MyDrive/Machine Visual Perception/3D-GS/gaussian-splatting/colab
fatal: destination path '/content/drive/MyDrive/Machine Visual Perception/3D-GS/gaussian-splatting/colab/splat' already exists and is not an empty directory.
--2024-11-28 14:17:22--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2024.11.1/cloudflared-linux-amd64.deb [following]
--2024-11-28 14:17:22--  https://github.com/cloudflare/cloudflared/releases/download/2024.11.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/

In [ ]:
# prompt: use splat/convert.py to convert the .ply in gaussian-splatting/output/... and then run this as the previous cell is doing (do the rest of the code), that is the most important thing
%cd $base_path
print(os.getcwd())
import os

# !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -O $base_path/cloudflared-linux-amd64.deb
# !dpkg -i $base_path/cloudflared-linux-amd64.deb

import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
%cd $base_path/splat
output = f"{base_path}/gaussian-splatting/output"
!python convert.py $output/73b9c001-a/point_cloud/iteration_7000/point_cloud.ply

#The rest of your code
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['cloudflared', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(7860, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)
%cd $base_path/splat
!python -m http.server 7860